# Berechnungen für Bayesianische Statistik im A/B Testing

---

Daten:
10000 Beobachtungen

4990 in Gruppe X mit 
720 x Erfolg und 
4270 x Nicht-Erfolg

5010 in Gruppe Y mit
817 x Erfolg und 
4193 x Nicht-Erfolg

Annahmen aus vergangenen Marketingkampagnen: zwischen 14 und 15% Konversationsrate
    

In [ ]:
# Imports packages
import numpy as np # for calculations
import pandas as pd # for printing dataframes
import matplotlib.pyplot as plt # for plotting
from matplotlib import rcParams
import seaborn as sns # for scientific plotting
import pymc3 as pm # for calculations with Bayes

In [ ]:
# Bestimmung passender Parameter für Alpha und Beta passend zhur Prioriverteilung
alpha = 10 # Anzahl der Erfolge
beta = 59 # Anzahl der Nicht-Erfolge
print(alpha/(alpha + beta))

prior_draws = np.random.beta(alpha, beta, 10000) # Dies erzeugt 10.000 zufällige Ziehungen innerhalb einer Beta-Verteilung mit den oben genannten Alpha- und Beta-Parametern, die als Prior-Verteilung verwendet wird

In [ ]:
# Priori-Verteilung plotten
plt.title("Priori-Annahmen der Konvertierungsrate")
sns.kdeplot(prior_draws, shade=True, label="Prioriverteilung")
rcParams['figure.figsize'] = 11.7,8.27 # Größe in Inches
plt.legend()
plt.xlabel("Konvertierungsrate in %")
plt.xticks(np.arange(0, max(prior_draws), 0.05))
plt.show()

In [ ]:
ads = pd.read_csv("daten_onlineshop_abtest_2021.csv", sep=";")
ads.head(5)

In [ ]:
# Data Cleaning
x_clicked = ads.loc[ads["site_version"] == "X"]["banner_clicked"] # Extrahiert die Spalte banner_clicked für jede Variante in Spalte site_version
y_clicked = ads.loc[ads["site_version"] == "Y"]["banner_clicked"]

In [ ]:
def simulate_beta_posterior(trials, beta_prior_a, beta_prior_b):
    ###
    ### Erzeugt eine Posterioriverteilung aus den Daten und der Prioriannahmen
    ###
    num_successes = np.sum(trials)   
    posterior_draws = np.random.beta(num_successes + beta_prior_a,len(trials) - num_successes + beta_prior_b, 100000)
    
    return posterior_draws

In [ ]:
# Simuliert Posterioriverteilungen
x_posterior = simulate_beta_posterior(x_clicked, alpha, beta)
y_posterior = simulate_beta_posterior(y_clicked, alpha, beta)

In [ ]:
# Die berechneten Intervallgrenzen der Posterioriverteilungen
pi_x1 = 0.134688
pi_x2 = 0.153892
pi_y1 = 0.152635
pi_y2 = 0.173004

In [ ]:
# Plottet die Posterioriverteilungen mit Kredibilitätsintervallen
sns.kdeplot(x_posterior, shade=True, label="Gruppe X")
sns.kdeplot(y_posterior, shade=True, label="Gruppe Y")
plt.title("Posterioriverteilung der Konvertierungsrate mit 95%-Kredibilitätsintervall")
plt.xlabel("Konvertierungsrate in %")
plt.vlines(pi_x1, 0, 80, linestyle='--', color='steelblue')
plt.vlines(pi_x2, 0, 80, linestyle='--', color='steelblue')
plt.vlines(pi_y1, 0, 80, linestyle='--', color='peru')
plt.vlines(pi_y2, 0, 80, linestyle='--', color='peru')
plt.legend()
plt.show()

In [ ]:
# Die berechneten Intervallgrenzen der Diferenz der Posterioriverteilungen
d1 = 0.00779
d2 = 0.02927

In [ ]:
# Berechnet die Posteriori-Differenzen und plottet sie
diff = y_posterior - x_posterior 
sns.kdeplot(diff, shade=True, label="Differenz in %")
plt.title("Verteilung der bayes. Differenz Gruppe Y - Gruppe X mit 95%-Kredibilitätsintervall")
plt.xlabel("Differenz in %")

plt.vlines(d1, 0, 55, linestyle='--', color='steelblue')
plt.vlines(d2, 0, 55, linestyle='--', color='steelblue')

plt.legend()
plt.show()

In [ ]:
# Plottet die frequentistische Verteilung mit Konfidenzintervallen
sns.kdeplot(np.random.normal(loc=0.144297292,scale=0.004898979,size=1000), shade=True,label="Gruppe X")
sns.kdeplot(np.random.normal(loc=0.162827328,scale=0.005196152,size=1000), shade=True,label="Gruppe Y")
plt.title("Frequentistische Verteilung der Konvertierungsrate mit 95%-Konfidenzintervall")
plt.xlabel("Konvertierungsrate in %")
plt.vlines(pi_x1, 0, 80, linestyle='--', color='steelblue')
plt.vlines(pi_x2, 0, 80, linestyle='--', color='steelblue')
plt.vlines(pi_y1, 0, 80, linestyle='--', color='peru')
plt.vlines(pi_y2, 0, 80, linestyle='--', color='peru')
plt.legend()
plt.show()

In [ ]:
d1 = 0.00464851
d2 = 0.03291149

In [ ]:
#Plottet die Differenz der frequentistischen Verteilungen mit Konfidenzintervallen
sns.kdeplot(np.random.normal(loc=0.01853,scale=0.00173205,size=1000), shade=True,label="Differenz in %")
plt.title("Frequentistische Verteilung der Differenz zwischen Gruppe Y und Gruppe X mit 95%-Konfidenzintervall")
plt.xlabel("Differenz in %")
plt.vlines(d1, 0, 80, linestyle='--', color='steelblue')
plt.vlines(d2, 0, 80, linestyle='--', color='steelblue')
plt.legend()
plt.show()